In [1]:
import pandas as pd 
from difflib import SequenceMatcher as SM
df_club=pd.read_csv("../Datasets/Clubs_data.csv")
df_meta=pd.read_csv("../Datasets/Metadata.csv")
df_organiser=pd.read_csv("../Datasets/Organisers_In_Fests.csv")
df_participants=pd.read_csv("../Datasets/Participants_In_Fests.csv")

In [2]:
def is_similarity(first, second, ratio):
    return SM(None, first, second).ratio() > ratio

In [3]:
#For Club name correction
for Index, row in df_club.iterrows():
    # print(Index, len(df_club.iloc[Index]["Name"]))
    df_club.at[Index, "Name"] = row["Name"].lower()
    for Index2, row2 in df_meta.iterrows():
        df_meta.at[Index2, "Name"] = row2["Name"].lower()
        if is_similarity(row["Name"], row2["Name"], 0.64):
            df_club.at[Index, "Name"] = row2["Name"]
            

In [4]:
#Removing Invalid Names in Clubs CSV
list_small_5 = []
for Index, row in df_club.iterrows():
    if len(df_club.iloc[Index]["Name"]) < 7:
        list_small_5.append(Index)
df_club = df_club.drop(df_club.index[list_small_5])

In [5]:
#For Participants Name correction
for Index, row in df_participants.iterrows():
    df_participants.at[Index, "Name"] = row["Name"].lower()
    for Index2, row2 in df_meta.iterrows():
        df_meta.at[Index2, "Name"] = row2["Name"].lower()
        if is_similarity(row["Name"], row2["Name"], 0.59):
            df_participants.at[Index, "Name"] = row2["Name"]

In [6]:
#Removing Invalid Names in Participations CSV
list_small_7 = []
for Index, row in df_participants.iterrows():
    if len(df_participants.iloc[Index]["Name"]) < 7:
        list_small_7.append(Index)
df_participants = df_participants.drop(df_participants.index[list_small_7])

In [7]:
#For Organisers Name Correction
for Index, row in df_organiser.iterrows():
    df_organiser.at[Index, "Name"] = row["Name"].lower()
    for Index2, row2 in df_meta.iterrows():
        df_meta.at[Index2, "Name"] = row2["Name"].lower()
        if is_similarity(row["Name"], row2["Name"], 0.69):
            df_organiser.at[Index, "Name"] = row2["Name"]

In [8]:
#Removing Invalid Names in Organisers CSV
list_small_7 = []
for Index, row in df_organiser.iterrows():
    if len(df_organiser.iloc[Index]["Name"]) < 7:
        list_small_7.append(Index)
df_organiser = df_organiser.drop(df_organiser.index[list_small_7])

In [9]:
#Importing Numpy and creating a Combined DF with all the data
import numpy as np
df_combined = pd.concat([df_club,df_organiser,df_participants], ignore_index=True)
df_combined['Roll_No'] = np.nan

In [10]:
#Assigning Roll No. to people for making access easier
for Index, row in df_combined.iterrows():
    for Index2, row2 in df_meta.iterrows():
        if row["Name"] == row2["Name"]:
            df_combined.at[Index, "Roll_No"] = row2["ID"]
df_combined = df_combined.sort_values(by="Roll_No")
df_combined_No_Null = df_combined[df_combined["Roll_No"].notnull()]

In [11]:
new_cols = ["Roll_No", "Name", "Club_Name", "Fest_Name" ,"Event", "Role"]
df_combined_No_Null=df_combined_No_Null.reindex(columns=new_cols)

In [12]:
[
    {
        'name': 'Dummy',
        'id': 'id000',
        'clubs': {
            'club_i': {
                'isOrganiser': 'Organiser',
                'club_i_event_j': {
                    'participated': False
                    },
            },
        },
        'fests': {
            'fest_i': {
                'isOrganiser': '',
                'fest_i_event_j': {
                    'participated': True
                    },
            },
        }
    }
]

[{'name': 'Dummy',
  'id': 'id000',
  'clubs': {'club_i': {'isOrganiser': 'Organiser',
    'club_i_event_j': {'participated': False}}},
  'fests': {'fest_i': {'isOrganiser': '',
    'fest_i_event_j': {'participated': True}}}}]

In [13]:
df_combined_No_Null.head()

,Roll_No,Name,Club_Name,Fest_Name,Event,Role
1704,17XJ1A0100,monet wallace,NaN,fest_2,fest_2_event_5,NaN
1395,17XJ1A0100,monet wallace,NaN,fest_2,fest_2_event_3,NaN
931,17XJ1A0100,monet wallace,club_2,NaN,club_2_event_1,Participant
2855,17XJ1A0100,monet wallace,NaN,fest_1,fest_1_event_1,NaN
4951,17XJ1A0100,monet wallace,NaN,fest_1,fest_1_event_4,NaN


In [35]:
import json
class dictonary(dict):
    def __init__(self):
        self = dict()

    def add(self, key, value):
        self[key] = value

In [33]:
json_profiles = []
for i in df_meta.index:
    name = dictonary()
    name.add("Name", df_meta["Name"][i])
    name.add("Id", df_meta["ID"][i])
    club = dictonary()
    for j in df_club.index:
        club_i = dictonary()
        if (is_similarity(df_meta["Name"][i], df_club["Name"][j], 0.65)):
            role = "" if df_club["Role"][j][0]=="P" else df_club["Role"][j]
            club_i_event_j = dictonary()
            club_i_event_j.add("Participated", (True if role=="" else False))
            club_i.add("isOrganiser", role)
            club_i.add(df_club["Event"][j], club_i_event_j)
            club_info = df_club["Event"][j]
            club_no = int(club_info[5:club_info.find("_",5)])
            event_no = int(club_info[club_info.rfind("_")+1:])
            club.add(df_club["Club_Name"][j],club_i)
    name.add("Clubs", club)
    fest = dictonary()
    for j in df_participants.index:
        fest_i = dictonary()
        if(is_similarity(df_meta["Name"][i], df_participants["Name"][j], 0.65)):
            role = ""
            fest_i_event_j = dictonary()
            for k in df_organiser.index:
                if(df_meta["Name"][i]==df_organiser["Name"][k]):
                    role = df_organiser["Role"][k]
                    break
            fest_i.add("isOrganiser", df_organiser["Role"][k])
            fest_i_event_j.add("Participated", (True if role=="" else False))
            fest_i.add(df_participants["Event"][j], fest_i_event_j)
            fest.add(df_participants["Fest_Name"][j], fest_i)
            fest_info = df_participants["Event"][j]
            fest_no = int(fest_info[5:fest_info.find("_", 5)])
            event_no = int(fest_info[fest_info.rfind("_")+1:])
    name.add("Fests", fest)
    json_profiles.append(name)

In [36]:
with open("../profiles.json", 'w') as f:
    json.dump(json_profiles, f)